<a href="https://colab.research.google.com/github/KoMurase/Learning_my_interest/blob/master/PyTorch_Chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chapter6　推薦システムと行列分解

In [9]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install tqdm

     |████████████████████████████████| 484.0MB 1.1MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.4.0 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
     |████████████████████████████████| 734.6MB 8.6kB/s 
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0
     |████████████████████████████████| 484.0MB 1.2MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.4.0 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


  Using cached https://files.pythonhosted.org/packages/88/95/90e8c4c31cfc67248bf944ba42029295b77159982f532c5689bcfe4e9108/torch-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0


In [10]:
import torch
torch.tensor([1,2,3]).to("cuda:0")

tensor([1, 2, 3], device='cuda:0')

tensor([1, 2, 3], device='cuda:0')

Colaboratory における圧縮ファイルの展開

In [11]:
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip ml-20m.zip

--2019-12-29 21:51:52--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  59.0MB/s    in 3.5s    

2019-12-29 21:51:56 (53.9 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         
--2019-12-29 21:54:45--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.group

KeyboardInterrupt: ignored

In [0]:
!ls

ml-20m	ml-20m.zip  sample_data


In [0]:
!ls ml-20m

genome-scores.csv  links.csv   ratings.csv  tags.csv
genome-tags.csv    movies.csv  README.txt


リスト6.1　DatasetとDataLoaderの作成（解凍したraiting.csvのパスを変更している）

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

In [0]:
import pandas as pd
# 訓練データとテストデータを分けるのに使用する
from sklearn import model_selection
df = pd.read_csv("ml-20m/ratings.csv")
# Xは(userId, movieId)のペア
X = df[["userId", "movieId"]].values
Y = df[["rating"]].values
# 訓練データとテストデータを9:1に分割
train_X, test_X, train_Y, test_Y\
    = model_selection.train_test_split(X, Y, test_size=0.1)
# XはIDで整数なのでint64, Yは実数値なのでfloat32のTensorに変換する
train_dataset = TensorDataset(
    torch.tensor(train_X, dtype=torch.int64), torch.tensor(train_Y, dtype=torch.float32))
test_dataset = TensorDataset(
    torch.tensor(test_X, dtype=torch.int64), torch.tensor(test_Y, dtype=torch.float32))
train_loader = DataLoader(
    train_dataset, batch_size=1024, num_workers=4, shuffle=True)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=4)

リスト6.2　行列因子分解

In [0]:
class MatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item, k=20):
        super().__init__()
        self.max_user = max_user
        self.max_item = max_item
        self.user_emb = nn.Embedding(max_user, k, 0)
        self.item_emb = nn.Embedding(max_item, k, 0)

    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)

        # user_feature*item_featureは(batch_size,k)次元なので
        # kについてsumをとるとそれぞれのサンプルの内積になる
        out = torch.sum(user_feature * item_feature, 1)

        # [0, 5]の範囲に収まるように変換
        out = nn.functional.sigmoid(out) * 5
        return out

リスト6.3　ユーザーや商品の個数

In [0]:
max_user, max_item = X.max(0)
# np.int64型をPythonの標準のintにキャスト
max_user = int(max_user)
max_item = int(max_item)
net = MatrixFactorization(max_user+1, max_item+1)

リスト6.4　評価関数の作成

In [0]:
def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
    ys = []
    ypreds = []
    for x, y in loader:
        x = x.to(device)
        ys.append(y)
        with torch.no_grad():
        	ypred = net(x).to("cpu").view(-1)
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score.item()

リスト6.5　訓練部分の作成

In [14]:
from statistics import mean

net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

for epoch in range(5):
    loss_log = []
    for x, y in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        o = net(x)
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score, flush=True)

  0%|          | 0/17579 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
 17%|█▋        | 2930/17579 [00:28<02:15, 108.31it/s]

KeyboardInterrupt: ignored

リスト6.6　ユーザー1の映画10に対する評価を実際に予測する場合

In [0]:
# 訓練したモデルをCPUに移す
net.to("cpu")

# ユーザー1の映画10の評価を計算したい
query = (1, 10)

# int64のTensorに変換し、batchの次元を付加
query = torch.tensor(query, dtype=torch.int64).view(1, -1)

# netに渡す
net(query)

リスト6.7　ユーザー1の上位5 本の映画をピックアップ

In [0]:
query = torch.stack([
    torch.zeros(max_item).fill_(1),
    torch.arange(1, max_item+1)
], 1).long()

# scoresは上位k本のスコア
# indicesは上位k本の位置、すなわちmovieId
scores, indices = torch.topk(net(query), 5)

リスト6.8　非線形の関数を使用してモデリングする

In [0]:
class NeuralMatrixFactorization(nn.Module):
    def __init__(self, max_user, max_item,
                 user_k=10, item_k=10,
                 hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            nn.Linear(user_k + item_k, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # ユーザー特徴量と商品特徴量をまとめて1つのベクトルにする
        out = torch.cat([user_feature, item_feature], 1)
        # まとめた特徴量ベクトルをMLPに入れる
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()


        ###

リスト6.9　区切られたジャンルをBoWに変換（<your_path>を変更している）

In [0]:
import csv
from sklearn.feature_extraction.text import CountVectorizer

# csv.DictReaderを使用してCSVファイルを読み込み
# 必要な部分だけ抽出する
with open("ml-20m/movies.csv") as fp:
    reader = csv.DictReader(fp)
    def parse(d):
        movieId = int(d["movieId"])
        genres = d["genres"]
        return movieId, genres
    data = [parse(d) for d in reader]

movieIds = [x[0] for x in data]
genres = [x[1] for x in data]

# データに合わせてCountVectorizerを訓練する
cv = CountVectorizer(dtype="f4").fit(genres)
num_genres = len(cv.get_feature_names())

# keyがmovieIdでvalueがBoWのTensorのdictを作る
it = cv.transform(genres).toarray()
it = (torch.tensor(g, dtype=torch.float32) for g in it)
genre_dict = dict(zip(movieIds, it))

リスト6.10　カスタムDatasetの作成

In [0]:
def first(xs):
    it = iter(xs)
    return next(it)

class MovieLensDataset(Dataset):
    def __init__(self, x, y, genres):
        assert len(x) == len(y)
        self.x = x
        self.y = y
        self.genres = genres
        
        # ジャンル辞書にないmovieIdの時のダミーデータ
        self.null_genre = torch.zeros_like(
            first(genres.values()))
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        # x = (userId, movieId)
        movieId = x[1]
        g = self.genres.get(movieId, self.null_genre)
        return x, y, g

リスト6.11　DataLoaderの作成

In [0]:
train_dataset = MovieLensDataset(
    torch.tensor(train_X, dtype=torch.int64),
    torch.tensor(train_Y, dtype=torch.float32), 
    genre_dict)
test_dataset = MovieLensDataset(
    torch.tensor(test_X, dtype=torch.int64),
    torch.tensor(test_Y, dtype=torch.float32),
    genre_dict)
train_loader = DataLoader(
    train_dataset, batch_size=1024, shuffle=True, num_workers=4)
test_loader = DataLoader(
    test_dataset, batch_size=1024, num_workers=4)

リスト6.12　ネットワークモデルの作成

In [0]:
class NeuralMatrixFactorization2(nn.Module):
    def __init__(self, max_user, max_item, num_genres,
                 user_k=10, item_k=10, hidden_dim=50):
        super().__init__()
        self.user_emb = nn.Embedding(max_user, user_k, 0)
        self.item_emb = nn.Embedding(max_item, item_k, 0)
        self.mlp = nn.Sequential(
            # num_genres分だけ次元が増える
            nn.Linear(user_k + item_k + num_genres, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, x, g):
        user_idx = x[:, 0]
        item_idx = x[:, 1]
        user_feature = self.user_emb(user_idx)
        item_feature = self.item_emb(item_idx)
        # ジャンルのBoWをcatで特徴ベクトルに結合する
        out = torch.cat([user_feature, item_feature, g], 1)
        out = self.mlp(out)
        out = nn.functional.sigmoid(out) * 5
        return out.squeeze()

リスト6.13　ヘルパー関数の修正

In [0]:
def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
    ys = []
    ypreds = []
    # loaderはジャンルのBoWも返す
    for x, y, g in loader:
        x = x.to(device)
        g = g.to(device)
        ys.append(y)
        # userId, movieId以外にジャンルの
        # BoWもネットワーク関数に渡す
        with torch.no_grad():
        	ypred = net(x, g).to("cpu")
        ypreds.append(ypred)
    score = score_fn(torch.cat(ys).squeeze(), torch.cat(ypreds))
    return score

リスト6.14　訓練部分の作成

In [21]:
net = NeuralMatrixFactorization2(
    max_user+1, max_item+1, num_genres)
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

net.to("cuda:0")
for epoch in range(5):
    loss_log = []
    net.train()
    for x, y, g in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        g = g.to("cuda:0")
        o = net(x, g)
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
    net.eval()
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score.item(), flush=True)


  0%|          | 0/17579 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

100%|██████████| 17579/17579 [04:44<00:00, 61.84it/s]

0 0.757757800008136 0.644826352596283



100%|██████████| 17579/17579 [04:46<00:00, 61.31it/s]

1 0.6920719851920735 0.6330887675285339



100%|██████████| 17579/17579 [04:53<00:00, 59.99it/s]

2 0.6711399810015393 0.6297889947891235



100%|██████████| 17579/17579 [04:49<00:00, 60.74it/s]

3 0.6581532667064173 0.625413715839386



100%|██████████| 17579/17579 [04:52<00:00, 60.14it/s]

4 0.6416557987223606 0.62052983045578


リスト6.15　ユーザー100に対してそれぞれのジャンルを1つだけ含んだ映画のスコアを計算

In [23]:
# 指定した位置だけ1で残りが0のTensorを返す補助関数
def make_genre_vector(i, max_len):
    g = torch.zeros(max_len)
    g[i] = 1
    return g

query_genres = [make_genre_vector(i, num_genres)
    for i in range(num_genres)]
query_genres = torch.stack(query_genres, 1)

# num_genres分だけuserId=100とmovieId=0のTensorを作って結合する
query = torch.stack([
    torch.empty(num_genres, dtype=torch.int64).fill_(100),
    torch.empty(num_genres, dtype=torch.int64).fill_(0)
], 1)

# GPUに転送
query_genres = query_genres.to("cuda:0")
query = query.to("cuda:0")

# スコアを計算
net(query, query_genres)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([2.9117, 2.9139, 2.9129, 2.9138, 2.9144, 2.9135, 2.9110, 2.9105, 2.9128,
        2.9148, 2.9131, 2.9136, 2.9125, 2.9131, 2.9135, 2.9106, 2.9114, 2.9115,
        2.9106, 2.9095, 2.9138, 2.9117, 2.9116, 2.9127], device='cuda:0',
       grad_fn=<SqueezeBackward0>)